In [ ]:
#from mstrio.api.browsing import get_objects_from_quick_search
from mstr_robotics.report import cube
from mstr_robotics.read_out_prj_obj import read_gen
from mstr_robotics.prepare_AI_data import redis_mstr_json
from mstr_robotics.redis_db import redis_bi_analysis
from mstrio.connection import Connection
from mstr_robotics.mstr_classes import mstr_global
from mstr_robotics.json_compare import JSONComparator,compare_mstr_objects
from mstr_robotics._helper import msic
from mstr_robotics.redis_db import fetch_it_all
import json
import pandas as pd

i_read_gen=read_gen()
i_redis_mstr_json=redis_mstr_json()
i_JSONComparator=JSONComparator()
i_msic=msic()
i_mstr_global=mstr_global()
i_compare_mstr_objects=compare_mstr_objects()

redis_env_d={"mstr_dev":{
        "host":"redis-14995.c56.east-us.azure.redns.redis-cloud.com",
        "port":14995,
        "password":"KSvQA45EE8ZEJ3xcFvus96927CW56D8u",  
        "username":"default",  
        "decode_responses":True,
        "project_id":"B7CA92F04B9FAE8D941C3E9B7E0CD754",
        "redis_prefix":"mstr_dev",
        "mstr_env":"http://85.214.60.83:8080/MicroStrategyLibrary/api"
},
        "mstr_test":{
        "host":"redis-13092.c282.east-us-mz.azure.redns.redis-cloud.com",
        "port":13092,
        "password":"pzSq7pQas9TFd6gMhcypYC70HRdyyvRv",
        "username":"default",
        "decode_responses":True,
        "project_id":"00F3103E41FF3743A057C3B5313595B7",
        "redis_prefix":"mstr_test",
        "mstr_env":"http://85.214.60.83:8080/MicroStrategyLibrary/api"
}}
redis_con_d=redis_env_d["mstr_dev"]

json_diff_d_l=[
{"org_j_f":{"env":"http://85.214.60.83:8080/MicroStrategyLibrary/api","project_id":"B7CA92F04B9FAE8D941C3E9B7E0CD754","object_id":"B793B55011D3E4E51000E887EC6DE8A4"},     "comp_j_f":{"env":"http://85.214.60.83:8080/MicroStrategyLibrary/api","project_id":"F8AD06364B29E4FFA3AEC9A34E706A47","object_id":"A90DC09E4A4EFF57D57734B418EDB667"}},
{"org_j_f":{"env":"http://85.214.60.83:8080/MicroStrategyLibrary/api","project_id":"B7CA92F04B9FAE8D941C3E9B7E0CD754","object_id":"98EB31B54122FFB738E6E08A2F29421A"},     "comp_j_f":{"env":"http://85.214.60.83:8080/MicroStrategyLibrary/api","project_id":"F8AD06364B29E4FFA3AEC9A34E706A47","object_id":"98EB31B54122FFB738E6E08A2F29421A"}}
]

redis_mstr_d={"project_prefix":{"B7CA92F04B9FAE8D941C3E9B7E0CD754":"mstr_dev",
                                "F8AD06364B29E4FFA3AEC9A34E706A47":"mstr_test"}}

redis_con_d=redis_env_d["mstr_dev"]
i_redis_bi_analysis = redis_bi_analysis( 
    host=redis_con_d["host"],
    port=redis_con_d["port"],
    password=redis_con_d["password"],
    username=redis_con_d["username"],
    decode_responses=redis_con_d["decode_responses"]
)

def path_to_last_bracket(list_l):
    """
    Find the last element starting with '[' and remove it along with all subsequent elements.
    
    Args:
        list_l: List of elements (strings or other types)
    
    Returns:
        Modified list with elements removed from the last '[' onwards
    """
    # Find the index of the last element starting with '['
    last_bracket_index = -1
    
    for i in range(len(list_l) - 1, -1, -1):
        # Convert to string to check if it starts with '['
        if str(list_l[i]).startswith('['):
            last_bracket_index = i
            break
    
    # If found, slice the list up to (but not including) that index
    if last_bracket_index != -1:
        return list_l[:last_bracket_index]
    
    # If no element starting with '[' found, return original list
    return list_l



In [ ]:

user_path="..\\config\\user_d.json"
with open(user_path, 'r') as file:
    user_d = json.load(file)

conn_params =  user_d["conn_params"]
#conn_params["project_id"]=project_id
conn = Connection(**conn_params)
conn.headers['Content-type'] = "application/json"
load_d={"base_url":conn.base_url,"project_name":conn.project_name,"username":conn.username}

In [ ]:
env='http://85.214.60.83:8080/MicroStrategyLibrary/api'
org_project_id="B7CA92F04B9FAE8D941C3E9B7E0CD754"
comp_project_id="F8AD06364B29E4FFA3AEC9A34E706A47"

check_obj_id_l=[]
check_obj_id_l.append("98EB31B54122FFB738E6E08A2F29421A")
#check_obj_id_l.append("8D679D3C11D3E4981000E787EC6DE8A4")

redis_mstr_d={"project_prefix":{"B7CA92F04B9FAE8D941C3E9B7E0CD754":"mstr_dev",
                                "F8AD06364B29E4FFA3AEC9A34E706A47":"mstr_test"}}
org_prefix=redis_mstr_d["project_prefix"][org_project_id]
comp_prefix=redis_mstr_d["project_prefix"][comp_project_id]
conn.select_project(org_project_id)
json_diff_d_l=[]


In [ ]:
short_cut_folder_id="B2A671234A1B55E512EC3F83A24A150A"
comp_project_id="F8AD06364B29E4FFA3AEC9A34E706A47"
conn.select_project(comp_project_id)
all_obj_d_l=i_mstr_global.get_obj_from_sh_fold(conn,folder_id=short_cut_folder_id)
check_obj_id_l=[]
for obj in all_obj_d_l:
    check_obj_id_l.append(obj["id"])
check_obj_id_l

In [ ]:
for obj_id in check_obj_id_l:
    comp_d={}
    comp_d["org_j_f"]={"env":env,"project_id":org_project_id,"object_id":obj_id,"redis_key":i_compare_mstr_objects.bld_redis_key(conn,obj_id,org_prefix)}
    comp_d["comp_j_f"]={"env":env,"project_id":comp_project_id,"object_id":obj_id,"redis_key":i_compare_mstr_objects.bld_redis_key(conn,obj_id,comp_prefix)}
    json_diff_d_l.append(comp_d.copy())
org_root_object_id_l=[]
comp_root_object_id_l=[]

recursive_fg=True

for obj in json_diff_d_l:
    #print(obj.keys())
    org_root_object_id_l.append(obj["org_j_f"]["redis_key"])
    comp_root_object_id_l.append(obj["comp_j_f"]["redis_key"])

i_fetch_it_all=fetch_it_all(i_redis_bi_analysis)
all_org_objects_df = pd.DataFrame(i_fetch_it_all.fetch_all_objects_recursively(root_object_l=org_root_object_id_l,recursive_fg=recursive_fg))
all_comp_objects_df = pd.DataFrame(i_fetch_it_all.fetch_all_objects_recursively(root_object_l=comp_root_object_id_l,recursive_fg=recursive_fg))


In [ ]:
diff_d_l=i_compare_mstr_objects.compare_objects( all_org_objects_df, all_comp_objects_df)

i_cube=cube()
conn.select_project("F8AD06364B29E4FFA3AEC9A34E706A47")
read_out_cbe_fold_id="9981CB214C79D75EC342A781539A4E5A"
df=pd.DataFrame(diff_d_l)
obj_prp_rel_d_l=[{"df":df,"tbl_name":"obj_prp_rel_df", "update_policy":"Replace" }]
cube_display_difference_id="0EA37C2B447E10A0EAE8FE83314E21E0"
cube_display_difference_name="display_difference"
i_cube.upload_cube_mult_table(conn=conn,tbl_upd_dict=obj_prp_rel_d_l,mtdi_id=cube_display_difference_id,
                              cube_name=cube_display_difference_name,folder_id=read_out_cbe_fold_id,force=True)

In [ ]:
df.head(4)
diff_fin_d_l = []
diff_todo_d_l = []

# Create lookup dictionaries for better performance
todo_config = {
    "FACT": ("entryLevel", "relationships"),
    "DIMENSION_USER": ("attributes", "relationships"),
    "REPORT": ("dataSource.dataTemplate", "advancedProperties")
}

remove_config = {
    "REPORT": "dataSource.filter"
}

exclude_config = {
    "REPORT": "dataSource.filter.text"
}

for i, obj_diff in df.iterrows():
    if not isinstance(obj_diff["json_key_path"], str):
        continue
    
    current_subtype = obj_diff["obj_subtype"]
    current_path = obj_diff["json_key_path"]
    
    # Check todo paths
    if current_subtype in todo_config:
        if current_path.startswith(todo_config[current_subtype]):
            diff_todo_d = obj_diff.to_dict()
            diff_todo_d["todo_subtype"]=current_subtype
            diff_todo_d["path"]=todo_config[current_subtype]
            diff_todo_d_l.append(diff_todo_d.copy())
            continue
    
    # Check remove paths (with exclusions)
    if current_subtype in remove_config:
        if current_path.startswith(remove_config[current_subtype]):
            # Check if it should be excluded
            if (current_subtype in exclude_config and 
                current_path.startswith(exclude_config[current_subtype])):
                # This goes to finished list instead
                diff_fin_d = obj_diff.to_dict()
                diff_fin_d_l.append(diff_fin_d.copy())
            # If not excluded, it gets removed (do nothing)
            continue
    
    # Everything else goes to finished
    diff_fin_d = obj_diff.to_dict()
    diff_fin_d_l.append(diff_fin_d)



print(f"Todo items: {len(diff_todo_d_l)}")
print(f"Finished items: {len(diff_fin_d_l)}")

In [ ]:
new_diff_todo_d_l=[]
for d in diff_todo_d_l:
    new_diff_todo_d=d.copy()
    new_diff_todo_d["diff_path"]=path_to_last_bracket(d["path_list"])
    new_diff_todo_d_l.append(new_diff_todo_d)
    
diff_todo_df=pd.DataFrame(new_diff_todo_d_l)
grouped_df = diff_todo_df.groupby([
    'org_root_obj_key',
    'org_obj_key',
    'comp_root_obj_key',
    'comp_obj_key',
    'name',
    'obj_subtype',
    'diff_type',
    'todo_subtype',
    'path'
]).agg({
    'value1': lambda x: list(x),
    'value2': lambda x: list(x)
}).reset_index()
grouped_df

In [42]:
grouped_df[grouped_df["org_obj_key"]=="mstr_dev:FACT:7A4ABE9111D5AC6FC000D98A4CC5F24F"]

,org_root_obj_key,org_obj_key,comp_root_obj_key,comp_obj_key,name,obj_subtype,diff_type,todo_subtype,path,value1,value2,comparison_status
6,mstr_dev:ATTRIBUTE:8D679D4911D3E4981000E787EC6DE8A4,mstr_dev:FACT:7A4ABE9111D5AC6FC000D98A4CC5F24F,mstr_test:ATTRIBUTE:8D679D4911D3E4981000E787EC6DE8A4,mstr_test:FACT:7A4ABE9111D5AC6FC000D98A4CC5F24F,Cost,FACT,different,FACT,"(entryLevel, relationships)","[3F58258B437F5CE376CB658F89ADB3AB, Days to Ship, 8D679D3F11D3E4981000E787EC6DE8A4, Employee, 8D6...","[8D679D3F11D3E4981000E787EC6DE8A4, Employee, 8D679D4211D3E4981000E787EC6DE8A4, Item, 8D679D4611D...",only_in_value1
7,mstr_dev:ATTRIBUTE:8D679D4911D3E4981000E787EC6DE8A4,mstr_dev:FACT:7A4ABE9111D5AC6FC000D98A4CC5F24F,mstr_test:ATTRIBUTE:8D679D4911D3E4981000E787EC6DE8A4,mstr_test:FACT:7A4ABE9111D5AC6FC000D98A4CC5F24F,Cost,FACT,missing,FACT,"(entryLevel, relationships)","[{'objectId': '95FA2DC34EC9063FD6C400B4FA8570E0', 'subType': 'attribute', 'name': 'Phone Usage'}]",[None],only_in_value1
24,mstr_dev:DIMENSION:B793B55011D3E4E51000E887EC6DE8A4,mstr_dev:FACT:7A4ABE9111D5AC6FC000D98A4CC5F24F,mstr_test:DIMENSION:B793B55011D3E4E51000E887EC6DE8A4,mstr_test:FACT:7A4ABE9111D5AC6FC000D98A4CC5F24F,Cost,FACT,different,FACT,"(entryLevel, relationships)","[3F58258B437F5CE376CB658F89ADB3AB, Days to Ship, 8D679D3F11D3E4981000E787EC6DE8A4, Employee, 8D6...","[8D679D3F11D3E4981000E787EC6DE8A4, Employee, 8D679D4211D3E4981000E787EC6DE8A4, Item, 8D679D4611D...",only_in_value1
25,mstr_dev:DIMENSION:B793B55011D3E4E51000E887EC6DE8A4,mstr_dev:FACT:7A4ABE9111D5AC6FC000D98A4CC5F24F,mstr_test:DIMENSION:B793B55011D3E4E51000E887EC6DE8A4,mstr_test:FACT:7A4ABE9111D5AC6FC000D98A4CC5F24F,Cost,FACT,missing,FACT,"(entryLevel, relationships)","[{'objectId': '95FA2DC34EC9063FD6C400B4FA8570E0', 'subType': 'attribute', 'name': 'Phone Usage'}]",[None],only_in_value1
40,mstr_dev:DOCUMENT_DEFINITION:98EB31B54122FFB738E6E08A2F29421A,mstr_dev:FACT:7A4ABE9111D5AC6FC000D98A4CC5F24F,mstr_test:DOCUMENT_DEFINITION:98EB31B54122FFB738E6E08A2F29421A,mstr_test:FACT:7A4ABE9111D5AC6FC000D98A4CC5F24F,Cost,FACT,different,FACT,"(entryLevel, relationships)","[3F58258B437F5CE376CB658F89ADB3AB, Days to Ship, 8D679D3F11D3E4981000E787EC6DE8A4, Employee, 8D6...","[8D679D3F11D3E4981000E787EC6DE8A4, Employee, 8D679D4211D3E4981000E787EC6DE8A4, Item, 8D679D4611D...",only_in_value1
41,mstr_dev:DOCUMENT_DEFINITION:98EB31B54122FFB738E6E08A2F29421A,mstr_dev:FACT:7A4ABE9111D5AC6FC000D98A4CC5F24F,mstr_test:DOCUMENT_DEFINITION:98EB31B54122FFB738E6E08A2F29421A,mstr_test:FACT:7A4ABE9111D5AC6FC000D98A4CC5F24F,Cost,FACT,missing,FACT,"(entryLevel, relationships)","[{'objectId': '95FA2DC34EC9063FD6C400B4FA8570E0', 'subType': 'attribute', 'name': 'Phone Usage'}]",[None],only_in_value1
56,mstr_dev:REPORT_DEFINITION:01274D824D6798C5F5BBB3BD4D15BA2C,mstr_dev:FACT:7A4ABE9111D5AC6FC000D98A4CC5F24F,mstr_test:REPORT_DEFINITION:01274D824D6798C5F5BBB3BD4D15BA2C,mstr_test:FACT:7A4ABE9111D5AC6FC000D98A4CC5F24F,Cost,FACT,different,FACT,"(entryLevel, relationships)","[3F58258B437F5CE376CB658F89ADB3AB, Days to Ship, 8D679D3F11D3E4981000E787EC6DE8A4, Employee, 8D6...","[8D679D3F11D3E4981000E787EC6DE8A4, Employee, 8D679D4211D3E4981000E787EC6DE8A4, Item, 8D679D4611D...",only_in_value1
57,mstr_dev:REPORT_DEFINITION:01274D824D6798C5F5BBB3BD4D15BA2C,mstr_dev:FACT:7A4ABE9111D5AC6FC000D98A4CC5F24F,mstr_test:REPORT_DEFINITION:01274D824D6798C5F5BBB3BD4D15BA2C,mstr_test:FACT:7A4ABE9111D5AC6FC000D98A4CC5F24F,Cost,FACT,missing,FACT,"(entryLevel, relationships)","[{'objectId': '95FA2DC34EC9063FD6C400B4FA8570E0', 'subType': 'attribute', 'name': 'Phone Usage'}]",[None],only_in_value1
75,mstr_dev:TABLE:24C30AD611D5AEC9C000E38A4CC5F24F,mstr_dev:FACT:7A4ABE9111D5AC6FC000D98A4CC5F24F,mstr_test:TABLE:24C30AD611D5AEC9C000E38A4CC5F24F,mstr_test:FACT:7A4ABE9111D5AC6FC000D98A4CC5F24F,Cost,FACT,different,FACT,"(entryLevel, relationships)","[3F58258B437F5CE376CB658F89ADB3AB, Days to Ship, 8D679D3F11D3E4981000E787EC6DE8A4, Employee, 8D6...","[8D679D3F11D3E49810

In [44]:
def classify_list_differences(row):
    """
    Compare value1 and value2 with support for nested dicts and lists.
    Returns items that only exist in value1, only in value2, and what's different.
    """
    import json
    from typing import Any
    
    def make_comparable(obj: Any) -> str:
        """Convert any object to a comparable string representation."""
        if obj is None or (isinstance(obj, float) and pd.isna(obj)):
            return ""
        
        if isinstance(obj, (dict, list)):
            return json.dumps(obj, sort_keys=True, default=str)
        else:
            return str(obj)
    
    def get_items(val):
        """Extract items from value (handles single values, lists, dicts)."""
        if val is None or (isinstance(val, float) and pd.isna(val)):
            return []
        
        if isinstance(val, list):
            return [make_comparable(item) for item in val]
        elif isinstance(val, dict):
            return [make_comparable(val)]
        else:
            return [make_comparable(val)]
    
    items1 = get_items(row['value1'])
    items2 = get_items(row['value2'])
    
    set1 = set(items1)
    set2 = set(items2)
    
    only_in_1 = list(set1 - set2)
    only_in_2 = list(set2 - set1)
    
    # Parse back to original types
    def parse_items(string_list):
        result = []
        for s in string_list:
            if not s:
                continue
            try:
                result.append(json.loads(s))
            except:
                result.append(s)
        return result
    
    only_in_value1 = parse_items(only_in_1)
    only_in_value2 = parse_items(only_in_2)
    
    # Build difference description
    differences = []
    if only_in_value1:
        differences.append({
            'type': 'only_in_value1',
            'items': only_in_value1,
            'count': len(only_in_value1)
        })
    if only_in_value2:
        differences.append({
            'type': 'only_in_value2',
            'items': only_in_value2,
            'count': len(only_in_value2)
        })
    
    # Determine status
    if len(only_in_value1) > 0:
        status = 'only_in_value1'
    elif len(only_in_value2) > 0:
        status = 'only_in_value2'
    else:
        status = 'identical'
    
    return pd.Series({
        'only_in_value1': only_in_value1,
        'only_in_value2': only_in_value2,
        'differences': differences,
        'difference_summary': f"v1: {len(only_in_value1)} unique, v2: {len(only_in_value2)} unique",
        'comparison_status': status
    })


# Usage
comparison_result = grouped_df.apply(classify_list_differences, axis=1)
grouped_df = pd.concat([grouped_df, comparison_result], axis=1)

comparison_result

,only_in_value1,only_in_value2,differences,difference_summary,comparison_status
0,"[Phone Usage, 95FA2DC34EC9063FD6C400B4FA8570E0]","[8D679D4911D3E4981000E787EC6DE8A4, Payment Method]","[{'type': 'only_in_value1', 'items': ['Phone Usage', '95FA2DC34EC9063FD6C400B4FA8570E0'], 'count...","v1: 2 unique, v2: 2 unique",only_in_value1
1,[],"[{'name': 'Phone Usage', 'objectId': '95FA2DC34EC9063FD6C400B4FA8570E0', 'subType': 'attribute'}]","[{'type': 'only_in_value2', 'items': [{'name': 'Phone Usage', 'objectId': '95FA2DC34EC9063FD6C40...","v1: 0 unique, v2: 1 unique",only_in_value2
2,"[Phone Usage, 95FA2DC34EC9063FD6C400B4FA8570E0]","[8D679D4911D3E4981000E787EC6DE8A4, Payment Method]","[{'type': 'only_in_value1', 'items': ['Phone Usage', '95FA2DC34EC9063FD6C400B4FA8570E0'], 'count...","v1: 2 unique, v2: 2 unique",only_in_value1
3,[],"[{'name': 'Phone Usage', 'objectId': '95FA2DC34EC9063FD6C400B4FA8570E0', 'subType': 'attribute'}]","[{'type': 'only_in_value2', 'items': [{'name': 'Phone Usage', 'objectId': '95FA2DC34EC9063FD6C40...","v1: 0 unique, v2: 1 unique",only_in_value2
4,"[Phone Usage, 95FA2DC34EC9063FD6C400B4FA8570E0]","[8D679D4911D3E4981000E787EC6DE8A4, Payment Method]","[{'type': 'only_in_value1', 'items': ['Phone Usage', '95FA2DC34EC9063FD6C400B4FA8570E0'], 'count...","v1: 2 unique, v2: 2 unique",only_in_value1
5,[],"[{'name': 'Phone Usage', 'objectId': '95FA2DC34EC9063FD6C400B4FA8570E0', 'subType': 'attribute'}]","[{'type': 'only_in_value2', 'items': [{'name': 'Phone Usage', 'objectId': '95FA2DC34EC9063FD6C40...","v1: 0 unique, v2: 1 unique",only_in_value2
6,"[Days to Ship, 3F58258B437F5CE376CB658F89ADB3AB]","[Phone Usage, 95FA2DC34EC9063FD6C400B4FA8570E0]","[{'type': 'only_in_value1', 'items': ['Days to Ship', '3F58258B437F5CE376CB658F89ADB3AB'], 'coun...","v1: 2 unique, v2: 2 unique",only_in_value1
7,"[{'name': 'Phone Usage', 'objectId': '95FA2DC34EC9063FD6C400B4FA8570E0', 'subType': 'attribute'}]",[],"[{'type': 'only_in_value1', 'items': [{'name': 'Phone Usage', 'objectId': '95FA2DC34EC9063FD6C40...","v1: 1 unique, v2: 0 unique",only_in_value1
8,"[Phone Usage, 95FA2DC34EC9063FD6C400B4FA8570E0]","[8D679D4911D3E4981000E787EC6DE8A4, Payment Method]","[{'type': 'only_in_value1', 'items': ['Phone Usage', '95FA2DC34EC9063FD6C400B4FA8570E0'], 'count...","v1: 2 unique, v2: 2 unique",only_in_value1
9,[],"[{'name': 'Phone Usage', 'objectId': '95FA2DC34EC9063FD6C400B4FA8570E0', 'subType': 'attribute'}]","[{'type': 'only_in_value2', 'items': [{'name': 'Phone Usage', 'objectId': '95FA2DC34EC9063FD6C40...","v1: 0 unique, v2: 1 unique",only_in_value2


In [ ]:
diff_todo_d_l[0]["org_obj_def"] 
diff_todo_d_l[0]["path_list"]

# Example usage:
path_to_extract = remove_from_last_bracket(diff_todo_d_l[0]["path_list"])
path_to_extract

In [ ]:
def extract_json_by_path(json_obj, path_l):
    """
    Extract a sub-JSON from a nested structure using a dot-notation path.
    
    Args:
        json_obj: The JSON object (dict/list) to extract from
        path: Dot-notation path (e.g., "user.address.city" or "items[0].name")
    
    Returns:
        The extracted sub-JSON, or None if path doesn't exist
    """

    # Split path by dots

    current = json_obj
    
    for key in path_l:
        if current is None:
            return None
        
        # Handle array indices like "items[0]"
        if '[' in key and ']' in key:
            # Split into key and index
            key_name = key[:key.index('[')]
            index = int(key[key.index('[') + 1:key.index(']')])
            
            if key_name:
                current = current.get(key_name) if isinstance(current, dict) else None
            
            if isinstance(current, list) and 0 <= index < len(current):
                current = current[index]
            else:
                return None
        else:
            # Regular dict key access
            if isinstance(current, dict):
                current = current.get(key)
            else:
                return None
    
    return current


# Example usage with your data structure:
path_to_extract = remove_from_last_bracket(diff_todo_d_l[0]["path_list"])
print(path_to_extract)
extracted_json = extract_json_by_path(diff_todo_d_l[0]["org_obj_def"], path_to_extract)

print(f"Path: {path_to_extract}")
print(f"Extracted JSON: {extracted_json}")

In [ ]:
def compare_dict_lists(org_l, comp_l, key_field='id'):
    """
    Compare two lists of dictionaries based on a key field.
    
    Args:
        org_l: Original list of dicts
        comp_l: Comparison list of dicts
        key_field: Field to use as unique identifier
    
    Returns:
        dict with differences
    """
    # Create lookup dictionaries
    org_dict = {item[key_field]: item for item in org_l if key_field in item}
    comp_dict = {item[key_field]: item for item in comp_l if key_field in item}
    
    org_keys = set(org_dict.keys())
    comp_keys = set(comp_dict.keys())
    
    result = {
        'only_in_org': [org_dict[k] for k in (org_keys - comp_keys)],
        'only_in_comp': [comp_dict[k] for k in (comp_keys - org_keys)],
        'different': [],
        'common': []
    }
    
    # Check items that exist in both
    for key in (org_keys & comp_keys):
        if org_dict[key] == comp_dict[key]:
            result['common'].append(org_dict[key])
        else:
            result['different'].append({
                'key': key
            })
    
    return result


# Example with your data structure:
# Assuming diff_todo_d_l contains items with 'org_obj_def' and 'comp_obj_def'
org_l = [item.get('org_obj_def') for item in diff_todo_d_l]
comp_l = [item.get('comp_obj_def') for item in diff_todo_d_l]
org_l = i_JSONComparator.remove_no_interest_fields(org_l)
comp_l = i_JSONComparator.remove_no_interest_fields(comp_l)

compare_dict_lists(org_l, comp_l)
# If comparing at the same indices:


In [ ]:

diff_d_l=[]
for i,comp in df_filtered.iterrows():
    comp_det_d={}
    comp_det_d["org_root_obj_key"]=comp["root_obj_key_x"]
    comp_det_d["org_obj_key"]=comp["obj_key_x"]
    comp_det_d["comp_root_obj_key"]=comp["root_obj_key_y"]
    comp_det_d["comp_obj_key"]=comp["obj_key_y"]
    comp_det_d["org_obj_def"]=comp["definition_x"]
    comp_det_d["comp_obj_def"]=comp["definition_y"]
    differences = i_JSONComparator.compare_json_data(comp_det_d=comp_det_d,
                                                     json1=comp["definition_x"] 
                                                 , json2=comp["definition_y"] )
    if len (differences)>0:

        diff_d_l.extend(differences)
    diff_d_l.append(comp_det_d)

i_cube=cube()
conn.select_project("F8AD06364B29E4FFA3AEC9A34E706A47")
read_out_cbe_fold_id="9981CB214C79D75EC342A781539A4E5A"
df=pd.DataFrame(diff_d_l)
obj_prp_rel_d_l=[{"df":df,"tbl_name":"obj_prp_rel_df", "update_policy":"Replace" }]
cube_display_difference_id="0EA37C2B447E10A0EAE8FE83314E21E0"
cube_display_difference_name="display_difference"
i_cube.upload_cube_mult_table(conn=conn,tbl_upd_dict=obj_prp_rel_d_l,mtdi_id=cube_display_difference_id,
                              cube_name=cube_display_difference_name,folder_id=read_out_cbe_fold_id,force=True)



In [ ]:
from mstr_robotics.report import cube
i_cube=cube()
conn.select_project("F8AD06364B29E4FFA3AEC9A34E706A47")
read_out_cbe_fold_id="9981CB214C79D75EC342A781539A4E5A"
df=pd.DataFrame(diff_d_l)
obj_prp_rel_d_l=[{"df":df,"tbl_name":"obj_prp_rel_df", "update_policy":"Replace" }]
cube_display_difference_id="0EA37C2B447E10A0EAE8FE83314E21E0"
cube_display_difference_name="display_difference"
i_cube.upload_cube_mult_table(conn=conn,tbl_upd_dict=obj_prp_rel_d_l,mtdi_id=cube_display_difference_id,
                              cube_name=cube_display_difference_name,folder_id=read_out_cbe_fold_id,force=True)


In [ ]:
diff_d_l=[]
for comp in comp_base_objdef_list:
    comp_det_d={}
    comp_det_d["org_project_id"]=comp["org_j_f"]["project_id"]
    comp_det_d["org_object_id"]=comp["org_j_f"]["object_id"]
    differences = i_JSONComparator.compare_json_data(comp_det_d=comp_det_d,
                                                     json1=comp["org_obj_def_j"] 
                                                 , json2=comp["comp_obj_def_j"] )
    if len (differences)>0:

        diff_d_l.extend(differences)
pd.DataFrame(diff_d_l)

In [ ]:

mstr_test:TABLE:167D1CEF4D7151F8BFB13193AAB9B40B

In [ ]:
from openai import OpenAI
from mstr_robotics.user_RAG import perplexity
import os
i_perplexity=perplexity()

def call_perplexity( msg_t, sys_cont, temperature=0.1):
    client = OpenAI(
        api_key=os.environ.get("PERPLEXITY_API_KEY"),
        # Set your API key in environment variables
        base_url="https://api.perplexity.ai"
    )

    try:
        # Create chat completion request
        messages = [
            {
                "role": "system",
                "content": sys_cont
            },
            {"role": "user", "content": msg_t}
        ]
        # print(messages)
        response = client.chat.completions.create(
            model="sonar-pro",  # Official model name for Perplexity-API
            messages=messages,
            temperature=temperature
        )
    except:
        pass
    return response


In [ ]:
sys_cont="""You are a JSON comparison specialist. Your task is to compare two nested 
JSON files representing MicroStrategy BI Objects and output a structured list of dictionaries suitable for DataFrame analysis.

INSTRUCTIONS:
1. Compare all keys, values, and nested structures between JSON A and JSON B
2. For each difference found, create a dictionary entry with these exact keys:
   - "path": dot-notation path to the difference (e.g., "user.address.city")
   - "type": type of difference ("missing_key", "value_change", "type_mismatch", "added_key")
   - "json_a_key": key or element in in json_a (use null if missing)
   - "json_b_key": key or element in in json_b (use null if missing)
   - "json_a_value": value or element in in json_a (use null if missing)
   - "json_b_value": value or element in in json_b (use null if missing)
3. If you do not find any differences, just bring back a list, with one dict, without any values
OUTPUT FORMAT: Return only a valid JSON array of dictionaries, no explanations. Never include an additional comment to the formal output

EXAMPLE OUTPUT:
[
  {
    "path": "user.name",
    "type": "value_change", 
    "json_a_value": "John",
    "json_b_value": "Jane",
    "data_type_a": "string",
    "data_type_b": "string"
  }
]"""
sys_cont

In [ ]:
import ast
com_result_l=[]
for comp in comp_base_objdef_list:
    try:
        json1 = i_JSONComparator.remove_no_interest_fields(comp["org_obj_def_j"]).copy()
        json2 = i_JSONComparator.remove_no_interest_fields(comp["comp_obj_def_j"]).copy()
        json1 = i_JSONComparator.clean_json(json1)
        json2 = i_JSONComparator.clean_json(json2)

        msg_t= f" Hello Perplexity, please describe me the difference between object_1: {json1} and object_2:{json2}"

        response=call_perplexity( msg_t=msg_t, sys_cont=sys_cont, temperature=0.1)
        json_t = json.loads(response.json())
        json_t_cont = i_perplexity.extract_json(json_t["choices"][0]["message"]["content"])
        json_t_cont_s = i_perplexity.clean_json(json_t_cont)

        json_t_cont_d = ast.literal_eval(json_t_cont_s)

        com_result_l.append(json_t_cont_d)
    except Exception as err:
        print(err)
        com_result_l.append(json_t_cont)
        #print(comp)

com_result_l

In [ ]:
json_t

In [ ]:
# Simple approach to convert json_t_cont to list of dicts
import ast
import json

# Method 1: Using ast.literal_eval (if json_t_cont is a string representation)
try:
    if isinstance(json_t_cont, str):
        json_t_cont_d = ast.literal_eval(json_t_cont)
    else:
        json_t_cont_d = json_t_cont
    
    # Ensure it's a list
    if not isinstance(json_t_cont_d, list):
        json_t_cont_d = [json_t_cont_d]
        
    print(f"Converted to list with {len(json_t_cont_d)} items")
    print(json_t_cont_d)
    
except Exception as e:
    print(f"Method 1 failed: {e}")
    
    # Method 2: Using json.loads (if it's a JSON string)
    try:
        if isinstance(json_t_cont, str):
            json_t_cont_d = json.loads(json_t_cont)
        else:
            json_t_cont_d = json_t_cont
            
        # Ensure it's a list
        if not isinstance(json_t_cont_d, list):
            json_t_cont_d = [json_t_cont_d]
            
        print(f"Method 2 success: {len(json_t_cont_d)} items")
        print(json_t_cont_d)
        
    except Exception as e2:
        print(f"Method 2 also failed: {e2}")
        
        # Method 3: Manual parsing if needed
        if isinstance(json_t_cont, str):
            # Clean the string and try to parse manually
            cleaned = json_t_cont.strip()
            if cleaned.startswith('[') and cleaned.endswith(']'):
                try:
                    json_t_cont_d = eval(cleaned)  # Use with caution
                except:
                    json_t_cont_d = []
            else:
                json_t_cont_d = [cleaned]  # Wrap single item in list
        else:
            json_t_cont_d = []

# Final result
json_t_cont_d

In [ ]:
from mstr_robotics.report import cube
i_cube=cube()

read_out_cbe_fold_id="9981CB214C79D75EC342A781539A4E5A"
df=pd.DataFrame(json_t_cont_d)
obj_prp_rel_d_l=[{"df":df,"tbl_name":"obj_prp_rel_df", "update_policy":"Replace" }]
cube_display_difference_id=None
cube_display_difference_name="display_difference"
i_cube.upload_cube_mult_table(conn=conn,tbl_upd_dict=obj_prp_rel_d_l,mtdi_id=cube_display_difference_id,
                              cube_name=cube_display_difference_name,folder_id=read_out_cbe_fold_id,force=True)